In [1]:
import pandas as pd
import requests
from time import sleep
from bs4 import BeautifulSoup

##### Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

дата - заголовок - ссылка на материал

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [2]:
def get_habr_articles(search:list):
    '''Извлекает статьи из поисковой выдачи habr.com.
    search принимает список поисковых запросов в формате list'''
    
    articles = []
    for query in search:
        res = requests.get(f'https://habr.com/ru/search/?q={query}')
        soup = BeautifulSoup(res.text)
        articles += soup.find_all('article', class_='tm-articles-list__item')
        sleep(0.3)
    return articles

In [3]:
request_list = ['python', 'аналитик данных', 'машинное обучение']

In [4]:
habr_articles = get_habr_articles(request_list)

In [5]:
def articles_to_df(articles):
    '''Переводит список статей в data frame со столбцами, содержащими дату, заголовок и ссылку на материал'''
    
    habr_search = pd.DataFrame()

    for el in articles:
        date = el.find('time')['title']
        title = el.find('a', class_='tm-article-snippet__title-link')
        span = title.find('span').text
        title_link = el.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2')
        link = 'https://habr.com' + title_link.find('a')['href']
        row = {'date': date, 'title': span, 'link': link}
        habr_search = pd.concat([habr_search, pd.DataFrame([row])])
    return habr_search.reset_index(drop=True).drop_duplicates(subset=['title'])

In [6]:
articles_to_df(habr_articles).head(20)

,date,title,link
0,"2022-05-06, 06:00",Respect Validation на Python,https://habr.com/ru/post/664426/
1,"2021-12-13, 09:00",Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...
2,"2022-01-20, 18:37",Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...
3,"2020-04-21, 18:35","Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/
4,"2021-07-06, 13:29",Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...
5,"2022-01-13, 18:35",Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...
6,"2020-12-04, 21:03",Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/
7,"2022-03-08, 12:13",Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/
8,"2020-03-03, 13:22",... начале этого года Python сместил Java и ...,https://habr.com/ru/company/itsumma/news/t/490...
9,"2021-11-16, 16:09",EPAM разработала бесплатный курс по программир...,https://habr.com/ru/company/epam_systems/news/...


##### Дополнительная часть
Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
дата - заголовок - ссылка на материал - текст материала - количество лайков

In [7]:
def get_habr_articles(search:list, pages:int):
    '''Извлекает статьи из поисковой выдачи habr.com.
    search принимает список поисковых запросов в формате list
    pages принимает количество страниц поиска, с которых необходимо забрать статьи'''
    
    articles = []
    for query in search:
        for i in range(1, pages+1):
            res = requests.get(f'https://habr.com/ru/search/page{i}/?q={query}')
            soup = BeautifulSoup(res.text)
            articles += soup.find_all('article', class_='tm-articles-list__item')
            sleep(0.3)
    return articles

In [8]:
request_list = ['python', 'аналитик данных']

In [9]:
habr_articles = get_habr_articles(request_list, 2)

In [10]:
def articles_to_df(articles):
    '''Переводит список статей в data frame со столбцами, содержащими дату, заголовок, ссылку на материал,
    текст материала и рейтинг статьи'''
        
    habr_search = pd.DataFrame()
    
    for el in articles:
        date = el.find('time')['title']
        title = el.find('a', class_='tm-article-snippet__title-link')
        span = title.find('span').text
        title_link = el.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2')
        link = 'https://habr.com' + title_link.find('a')['href']
        
        # получение текста и количества лайков статьи из link
        ar_req = requests.get(link)
        sleep(0.3)
        ar_soup = BeautifulSoup(ar_req.text)
        ar_body = ar_soup.find('div', class_='tm-article-body')
        ar_text = ar_body.text.strip()
        ar_rating = ar_soup.find('div', class_='tm-article-sticky-panel')
        ar_likes = ar_rating.find('span').text
        
        row = {'date': date, 'title': span, 'link': link, 'text': ar_text, 'likes': ar_likes}
        habr_search = pd.concat([habr_search, pd.DataFrame([row])])
    return habr_search.reset_index(drop=True).drop_duplicates(subset=['title'])

In [11]:
articles_to_df(habr_articles).head(20)

,date,title,link,text,likes
0,"2022-05-06, 06:00",Respect Validation на Python,https://habr.com/ru/post/664426/,"В данной статье я расскажу вам историю ""как я ...",+19
1,"2021-12-13, 09:00",Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...,21 декабря Слёрм проведёт открытый урок «ChatO...,+9
2,"2022-01-20, 18:37",Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,"Курс нацелен дать максимальную пользу, поэтому...",+10
3,"2020-04-21, 18:35","Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,"20 апреля 2020 года, спустя почти десять лет п...",+19
4,"2021-07-06, 13:29",Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...,"Первый открытый онлайн-митап сообщества, для к...",+3
5,"2022-01-13, 18:35",Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...,19 января Слёрм проведёт открытый урок «Пишем ...,+10
6,"2020-12-04, 21:03",Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/,По данным широко известного в узких кругах Tio...,+16
7,"2022-03-08, 12:13",Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/,Вышел выпуск расширения Python для Visual Stud...,0
8,"2020-03-03, 13:22",... начале этого года Python сместил Java и ...,https://habr.com/ru/company/itsumma/news/t/490...,"Согласно отчету RedMonk за январь 2020 года, P...",+31
9,"2021-11-16, 16:09",EPAM разработала бесплатный курс по программир...,https://habr.com/ru/company/epam_systems/news/...,Компания EPAM запустила бесплатный курс для са...,+3
